In [1]:
import glob
import os
from osgeo import gdal
import numpy as np
import pandas as pd
import pylab as plt
import plotly.graph_objects as go
import h5py
from matplotlib.pyplot import *
import imageio
import rasterio

In [2]:
# Glob together all of the albedo datasets.
albedo = glob.glob('*.h5')
albedo

['SierraAlbedo2001.h5',
 'SierraAlbedo2002.h5',
 'SierraAlbedo2003.h5',
 'SierraAlbedo2004.h5',
 'SierraAlbedo2005.h5',
 'SierraAlbedo2006.h5',
 'SierraAlbedo2007.h5',
 'SierraAlbedo2008.h5',
 'SierraAlbedo2009.h5',
 'SierraAlbedo2010.h5',
 'SierraAlbedo2011.h5',
 'SierraAlbedo2012.h5',
 'SierraAlbedo2013.h5',
 'SierraAlbedo2014.h5',
 'SierraAlbedo2015.h5',
 'SierraAlbedo2016.h5',
 'SierraAlbedo2017.h5',
 'SierraAlbedo2018.h5',
 'SierraAlbedo2019.h5']

## Calculate Monthly Albedo Averages

In [3]:
# Get subdatasets of first snow fraction dataset ('SierraAlbedo2001.h5').
time_list = []
for i in range(len(albedo)):
    dataset = gdal.Open(albedo[i], gdal.GA_ReadOnly)


    #Changes the selected dataset into an array.
    albedo_array = dataset.ReadAsArray()

    albedo_float = albedo_array.astype('float')
    albedo_float[albedo_float == 65535] = np.nan
    albedo_float[albedo_float == 0] = np.nan


    albedo_test = np.transpose(albedo_float)
    
    # Make a variable for the starting year of each water year.
    year =  i + 2000
    # Creates a variable for the first date in the water year. 
    year_month = (str(year) + "-10-01")
    # Creates a list of datetimes based on each year in our dataset.
    year_month_date = pd.Series(pd.date_range((year_month), periods =(len(albedo_array)), freq="d"))
    #Need to create an empty list to append our mean values to. 
    new_list = []
    # For loop to calculate the mean for each month per year. 
    for j in range (1, 13):
        # Subset the date year based on month.
        month = year_month_date[year_month_date.dt.month == j]
        # Subset dataset by each month.
        month_len = albedo_test[:,:,(list(month.index))]
        # Take the mean of each month per year. 
        mean = np.mean(month_len, axis = 2)
        # Append mean values to list per year. 
        new_list.append(mean)
    # Append year lists to empty list. 
    time_list.append(new_list)
# Converts list to array. 
date_array = np.array(time_list)
np.shape(date_array)

# Since we're interested in albedo rates, we need to divide our values by the divisor(10000) 

# Create empty list to put values in 
month_divisor = []
# Select year
for i in range(len(date_array[:])):
    year_divisor = date_array[i]
    sub_list = []
    for j in range(len(year_divisor[:])):
        month_anom = year_divisor[j]/10000
        sub_list.append(month_anom)
    month_divisor.append(sub_list)
divisor_array = np.array(month_divisor)

## Calculate Annual Mean Albedo Percentage for Each Year

In [18]:
# Takes the mean of each monthly mean
# Note: to get true mean (weighted mean), we would need to take the mean of the cumulatice days of a single month, then divide by the number of days. 
# However, since the sample size only varies by one day every four years (leap years), this will give us near identical values. 
month_mean_albedo = np.mean(divisor_array, axis = 0)
np.shape(month_mean_albedo)

(12, 1841, 1334)

In [19]:
np.unique(month_mean_albedo)

array([0.567209  , 0.57156503, 0.57293718, ..., 0.81132615, 0.81249525,
              nan])

## Calculate Monthly Anomalies

In [5]:
# Create empty list to put values in 
monthly_anom = []
# Select year
for i in range(len(divisor_array[:])):
    year_anom = divisor_array[i]
    sub_list = []
    for j in range(len(year_anom[:])):
        month_anom = year_anom[j] - month_mean_albedo[j]
        sub_list.append(month_anom)
    monthly_anom.append(sub_list)
anom_array = np.array(monthly_anom)

## Select Indivdual Years and Months

In [6]:
# Convert our 3d mean annual list to array then select a single year.
year_array = np.array(month_divisor)
year_one_mean = year_array[0,:,:,]
year_three_mean = year_array[2,:,:,]
np.shape(year_one_mean)

(12, 1841, 1334)

In [7]:
year_one_anom = anom_array[0]
year_one_anom = np.transpose(year_one_anom)

In [8]:
year_one_month_one_anom = anom_array[0][0]

In [9]:
# Converts list to array. 
date_array = np.array(time_list)

In [10]:
#Subset our data to select year one (0 in this case)
year_one = date_array[0,:,:,:]

In [11]:
# Transpose data so that the dataset is ordered as: [xdim, ydim, month]
year_one_transposed = np.transpose(year_one, (1,2,0))
np.shape(year_one_transposed)

(1841, 1334, 12)

In [12]:
year_one_month_one = year_one_transposed[:,:,0]
np.shape(year_one_month_one)

(1841, 1334)

## Create Gif of a Single Year

In [ ]:
    # Create an empty list.
    filenames = []
    
    for i in range (12):
        # 3rd dimension is day. 
        total_d = year_one_anom[:,:,i] 
        plt.figure(figsize=(10, 10))
        plt.imshow(total_d, interpolation = 'nearest')
        plt.title('MODIS_GRID_500m 2001 Snow Cover Day: ' + str(i), fontsize = 20, fontweight='bold')
        
        filename = f'{i}.png'
        filenames.append(filename)
        
        # save frame
        plt.savefig(filename)
        plt.close()
        
            
    with imageio.get_writer('year_2.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
    for filename in set(filenames):
        os.remove(filename)

## Convert Monthly Averages to Geotiffs

In [13]:
# x dimension of array
xdim = albedo_array.shape[1]
# y dimension of array
ydim = albedo_array.shape[2]
# Projection data of sample GeoTiff
projection = 'PROJCS["Albers Conical Equal Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
# transformation data of array
# Pull refrencing matrix from h5 file.
ref_matrix_meta = dataset.GetMetadata()['Grid_MODIS_GRID_500m_ReferencingMatrix'].split()
referencing_matrix = [int(ref_matrix_meta[2]), int(ref_matrix_meta[1]), int(ref_matrix_meta[0]), int(ref_matrix_meta[5]), int(ref_matrix_meta[4]), int(ref_matrix_meta[3])]

### Convert Single Layer to Geotiff

In [14]:
def SingleGeotiff(raster_name, data, height, width, geotransform, wkt):
    
    driver = gdal.GetDriverByName('GTiff')

    dataset = driver.Create(
        raster_name,
        width,
        height,
        1,
        gdal.GDT_Float32)

    dataset.SetGeoTransform((
     geotransform))

    dataset.SetProjection(wkt)
    dataset.GetRasterBand(1).WriteArray(data)
    dataset.FlushCache()  # Write to disk.
    return dataset, dataset.GetRasterBand(1) 

In [16]:
SingleGeotiff('Annual_Janurary.tif', month_mean_albedo[0], ydim, xdim, referencing_matrix, projection)

(<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000015FBA4F2E70> >,
 <osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x0000015FBA499450> >)

## Convert Stacked Array to Stacked Geotiff

In [ ]:
def StackedGeotiff(name, array, geo_transform, projection):
    
    driver = gdal.GetDriverByName('GTiff')

    DataSet = driver.Create(name, array.shape[2], array.shape[1], array.shape[0], gdal.GDT_Float32)
    DataSet.SetGeoTransform(geo_transform)
    DataSet.SetProjection(projection)
    for i, image in enumerate(array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
    DataSet.FlushCache()
    return name

In [ ]:
StackedGeotiff('year_one_stack.tif', year_one, referencing_matrix, projection)